In [35]:
from roboflow import Roboflow
from statistics import mean
import torchvision
from tqdm import tqdm
from torch.nn.functional import threshold, normalize
from torchvision.ops import masks_to_boxes
from torchvision.utils import draw_bounding_boxes
from tqdm import tqdm
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [36]:
import copy
from imantics import Polygons, Mask

In [37]:
device = 'cuda:0'
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
sam_model = sam_model_registry['vit_h'](checkpoint='samweigth/sam_vit_h_4b8939.pth')
sam_model.to(device)
predictor = SamPredictor(sam_model)

In [38]:
mask_generator = SamAutomaticMaskGenerator(sam_model)

In [39]:
path = ''
red_path = path+'test/'
green_path = path+'Auto_green/'
label_path =  path+'test_label/'

In [40]:
red_files = os.listdir(red_path)
green_files = os.listdir(green_path)
if '.ipynb_checkpoints' in red_files:
    red_files.remove('.ipynb_checkpoints')

In [41]:
len(red_files)

32

In [42]:
use_center_of_mass= False

In [ ]:

for jjjjj in [1]: # [2,3,4,5]
    # data_path = f'datasets_det{i}'
    # !mv $data_path datasets
    # !yolo task=detect mode=train model=yolov8x.pt data=./datasets/data.yaml epochs=100 imgsz=640 plots=True
    # !mv datasets $data_path 
    
    
    # This parameter is from your roboflow workshop project

    #yourAPI = XXXXX  GET API from roboflow
    rf = Roboflow(api_key=f"{yourAPI}")
    project = rf.workspace("capdetect").project(f"data_prep_det0")
    model = project.version(1).model
    #---------------------
    USE_SAM = True
    
    conf = 50
    over = 80

    
    # save_name = f'onlyred_conf{conf}_over{over}/'
    
    
    
    save_path_our = path+f'output_new/our/{jjjjj}/'
    save_path_box = path+f'output_new/box/{jjjjj}/'
    save_path_point = path+f'output_new/point/{jjjjj}/'
    save_path_sam = path+f'output_new/sam/{jjjjj}/'
    
    save_path_area = path+f'output_new/area/{jjjjj}/'
    
    os.makedirs(save_path_our,exist_ok = True)   #AutoQC
    os.makedirs(save_path_box,exist_ok = True)   #only BBOX
    os.makedirs(save_path_point,exist_ok = True) #only points
    os.makedirs(save_path_sam,exist_ok = True)  #only same
    # os.makedirs(save_path_area,exist_ok = True) 
    
    
    for i in range(len(red_files)):
    
        # print(i)
        ### prompt-based
        image_name = red_files[i].split('.')[0] #test_name[i]
        pred_label = {}
        pred_label['info'] = {}
        pred_label['info']['name'] = image_name+'.jpg'

        # image_name ='116_jpg.rf.36bd06d127f9d04216932cea04a15d48.jpg'
        image  = cv2.imread(red_path+image_name+'.jpg',-1)

        pred_label['info']['width'] = image.shape[0]
        pred_label['info']['height'] = image.shape[1]
        pred_label['info']['depth'] = image.shape[2]


        image2  = cv2.imread(green_path+image_name+'.tif',-1)
        # image  = cv2.imread(image_name,-1)
        boxes = model.predict(red_path+image_name+'.jpg', confidence=conf, overlap=over).json()
        # boxes = model.predict(image, confidence=50, overlap=50).json()
        predictor.set_image(image)

        prompt_points = []
        for predciton in boxes['predictions']:
            # Object = {}

            # Object['class '] = predciton['x']
            x1,y1,x2,y2 = predciton['x']-predciton['width']/2,predciton['y']-predciton['height']/2,predciton['x']+predciton['width']/2,predciton['y']+predciton['height']/2
            input_box= np.array([x1,y1,x2,y2])
            cate = predciton['class']
            # predictor.set_image(image)
            if use_center_of_mass== False:
                prompt_points.append([predciton['x'], predciton['y']])
            else:
                cx,cy = get_center_of_mass( input_box, image2)
                prompt_points.append([cx,cy])


        masks = []    
        masks_nosave = []
        skip_seg = []
        for j in range(len(prompt_points)):
            if j not in skip_seg:
                Object = {}

                # point_labels = np.zeros(len(prompt_points),dtype=int)
                # point_labels[j]=1
                predciton =  boxes['predictions'][j]
                x1,y1,x2,y2 = predciton['x']-predciton['width']/2,predciton['y']-predciton['height']/2,predciton['x']+predciton['width']/2,predciton['y']+predciton['height']/2
                input_box= np.array([x1,y1,x2,y2])
                Object['bbox'] =[x1,y1,x2,y2]

                point_labels = [1]
                prompt_points_temp = [prompt_points[j]]
                r0 = min(predciton['height'],predciton['width'])

                for k in range(len(prompt_points)):
                    if k!=j:
                        if  prompt_points[k][0]>x1 and prompt_points[k][0]<x2 and  prompt_points[k][1]>y1 and prompt_points[k][1]<y2:
                            # print('hehre')

                            r = min(boxes['predictions'][k]['height'],boxes['predictions'][k]['width'])

                            if boxes['predictions'][k]['class'] ==predciton['class']  and  \
                            (prompt_points[k][0]-prompt_points[j][0])**2+(prompt_points[k][1]-prompt_points[j][1])**2<min(r0,r)**2 *0.1**2 :
                                skip_seg.append(k)
                            else:
                                prompt_points_temp.append(prompt_points[k])
                                point_labels.append(0)



                mask, _, _ = predictor.predict(
                    point_coords=np.array(prompt_points_temp),
                    point_labels=np.array(point_labels),
                    box=input_box[None, :],
                    multimask_output=False,
                    )

                polygons = Mask(mask[0]).polygons()

                if len(polygons.points)>1:
                    lens = np.array([len(i) for i in polygons.points])
                    index = np.where(lens==max(lens))[0].item()
                    m = np.zeros(image.shape[:2])
                    img_origin = m.copy()
                    cv2.fillPoly(m,[np.array(polygons.points[index]).astype(np.int32)], 1)
                    m =  cv2.bitwise_xor(m, img_origin)
                    polygons_points = polygons.points[index].tolist()

                else:
                    m = np.zeros(image.shape[:2])
                    img_origin = m.copy()
                    cv2.fillPoly(m,[np.array(polygons.points[0]).astype(np.int32)], 1)
                    m =  cv2.bitwise_xor(m, img_origin)
                    polygons_points = polygons.points[0].tolist()

                Object['area'] = np.sum(m)   
                Object['score'] =predciton['confidence']
                Object['segmentation'] = polygons_points
                Object['category'] =predciton['class']
                masks.append(Object)
                Object_temp = copy.deepcopy(Object)
                Object_temp['segmentation_sam'] = m
                masks_nosave.append(Object_temp)
        pred_label['objects'] = masks
        #save to file ...
        pred_label_save = copy.deepcopy(pred_label)

        pred_label = json.dumps(pred_label)
        f = open(os.path.join(save_path_our+f'{image_name}.json'), 'w')
        f.write(pred_label)
        f.close()

        ### only sam
        if USE_SAM== True:

            masks_o = mask_generator.generate(image)
            pred_label = {}
            pred_label['info'] = {}
            pred_label['info']['name'] = image_name+'.jpg'

            # image_name ='116_jpg.rf.36bd06d127f9d04216932cea04a15d48.jpg'
            image  = cv2.imread(red_path+image_name+'.jpg',-1)

            pred_label['info']['width'] = image.shape[0]
            pred_label['info']['height'] = image.shape[1]
            pred_label['info']['depth'] = image.shape[2]

            mask_sam = []
            for predciton in masks_o:

                Object = {}
                polygons = Mask(predciton['segmentation']).polygons()

                if len(polygons.points)>1:
                    lens = np.array([len(i) for i in polygons.points])
                    index = np.where(lens==max(lens))[0].item()
                    m = np.zeros(image.shape[:2])
                    img_origin = m.copy()
                    cv2.fillPoly(m,[np.array(polygons.points[index]).astype(np.int32)], 1)
                    m =  cv2.bitwise_xor(m, img_origin)
                    polygons_points = polygons.points[index].tolist()

                else:
                    m = np.zeros(image.shape[:2])
                    img_origin = m.copy()
                    cv2.fillPoly(m,[np.array(polygons.points[0]).astype(np.int32)], 1)
                    m =  cv2.bitwise_xor(m, img_origin)
                    polygons_points = polygons.points[0].tolist()
                Object['bbox'] = predciton['bbox']  
                Object['area'] = np.sum(m)   
                Object['score'] =predciton['stability_score']
                Object['segmentation'] = polygons_points
                if np.random.rand()>0.5:
                    Object['category'] =  'cell'   #predciton['class']  #need predict
                else:
                    Object['category'] =  'capillary'
                mask_sam.append(Object)
            pred_label['objects'] =   mask_sam  
            pred_label = json.dumps(pred_label)
            f = open(os.path.join(save_path_sam+f'{image_name}.json'), 'w')
            f.write(pred_label)
            f.close()
    # only box
    for i in range(len(red_files)):
    
        # print(i)
        ### prompt-based
        image_name = red_files[i].split('.')[0] #test_name[i]
        pred_label = {}
        pred_label['info'] = {}
        pred_label['info']['name'] = image_name+'.jpg'

        # image_name ='116_jpg.rf.36bd06d127f9d04216932cea04a15d48.jpg'
        image  = cv2.imread(red_path+image_name+'.jpg',-1)

        pred_label['info']['width'] = image.shape[0]
        pred_label['info']['height'] = image.shape[1]
        pred_label['info']['depth'] = image.shape[2]


        image2  = cv2.imread(green_path+image_name+'.tif',-1)
        # image  = cv2.imread(image_name,-1)
        
        boxes = model.predict(red_path+image_name+'.jpg', confidence=conf, overlap=over).json()
        # boxes = model.predict(image, confidence=50, overlap=50).json()
        predictor.set_image(image)

        prompt_points = []
        for predciton in boxes['predictions']:
            # Object = {}

            # Object['class '] = predciton['x']
            x1,y1,x2,y2 = predciton['x']-predciton['width']/2,predciton['y']-predciton['height']/2,predciton['x']+predciton['width']/2,predciton['y']+predciton['height']/2
            input_box= np.array([x1,y1,x2,y2])
            cate = predciton['class']
            # predictor.set_image(image)
            if use_center_of_mass== False:
                prompt_points.append([predciton['x'], predciton['y']])
            else:
                cx,cy = get_center_of_mass( input_box, image2)
                prompt_points.append([cx,cy])


        masks = []    
        masks_nosave = []
        skip_seg = []
        for j in range(len(prompt_points)):
            if True:
                Object = {}

                # point_labels = np.zeros(len(prompt_points),dtype=int)
                # point_labels[j]=1
                predciton =  boxes['predictions'][j]
                x1,y1,x2,y2 = predciton['x']-predciton['width']/2,predciton['y']-predciton['height']/2,predciton['x']+predciton['width']/2,predciton['y']+predciton['height']/2
                input_box= np.array([x1,y1,x2,y2])
                Object['bbox'] =[x1,y1,x2,y2]

                point_labels = [1]
                prompt_points_temp = [prompt_points[j]]
                r0 = min(predciton['height'],predciton['width'])

                # for k in range(len(prompt_points)):
#                     if k!=j:
#                         if  prompt_points[k][0]>x1 and prompt_points[k][0]<x2 and  prompt_points[k][1]>y1 and prompt_points[k][1]<y2:
#                             # print('hehre')

#                             r = min(boxes['predictions'][k]['height'],boxes['predictions'][k]['width'])

#                             if boxes['predictions'][k]['class'] ==predciton['class']  and  \
#                             (prompt_points[k][0]-prompt_points[j][0])**2+(prompt_points[k][1]-prompt_points[j][1])**2<min(r0,r)**2 *0.1**2 :
#                                 skip_seg.append(k)
#                             else:
#                                 prompt_points_temp.append(prompt_points[k])
#                                 point_labels.append(0)



                mask, _, _ = predictor.predict(
                    point_coords=None,
                    point_labels=None,
                    box=input_box[None, :],
                    multimask_output=False,
                    )


                polygons = Mask(mask[0]).polygons()

                if len(polygons.points)>1:
                    lens = np.array([len(i) for i in polygons.points])
                    index = np.where(lens==max(lens))[0].item()
                    m = np.zeros(image.shape[:2])
                    img_origin = m.copy()
                    cv2.fillPoly(m,[np.array(polygons.points[index]).astype(np.int32)], 1)
                    m =  cv2.bitwise_xor(m, img_origin)
                    polygons_points = polygons.points[index].tolist()

                else:
                    m = np.zeros(image.shape[:2])
                    img_origin = m.copy()
                    cv2.fillPoly(m,[np.array(polygons.points[0]).astype(np.int32)], 1)
                    m =  cv2.bitwise_xor(m, img_origin)
                    polygons_points = polygons.points[0].tolist()

                Object['area'] = np.sum(m)   
                Object['score'] =predciton['confidence']
                Object['segmentation'] = polygons_points
                Object['category'] =predciton['class']
                masks.append(Object)
                Object_temp = copy.deepcopy(Object)
                Object_temp['segmentation_sam'] = m
                masks_nosave.append(Object_temp)
        pred_label['objects'] = masks
        #save to file ...
        pred_label_save = copy.deepcopy(pred_label)

        pred_label = json.dumps(pred_label)
        f = open(os.path.join(save_path_box+f'{image_name}.json'), 'w')
        f.write(pred_label)
        f.close()
        
    # only point
    for i in range(len(red_files)):
    
        # print(i)
        ### prompt-based
        image_name = red_files[i].split('.')[0] #test_name[i]
        pred_label = {}
        pred_label['info'] = {}
        pred_label['info']['name'] = image_name+'.jpg'

        # image_name ='116_jpg.rf.36bd06d127f9d04216932cea04a15d48.jpg'
        image  = cv2.imread(red_path+image_name+'.jpg',-1)

        pred_label['info']['width'] = image.shape[0]
        pred_label['info']['height'] = image.shape[1]
        pred_label['info']['depth'] = image.shape[2]


        image2  = cv2.imread(green_path+image_name+'.tif',-1)
        # image  = cv2.imread(image_name,-1)
        # boxes = model.predict(red_path+image_name+'.jpg', confidence=50, overlap=80).json()
        boxes = model.predict(red_path+image_name+'.jpg', confidence=conf, overlap=over).json()
        # boxes = model.predict(image, confidence=50, overlap=50).json()
        predictor.set_image(image)

        prompt_points = []
        for predciton in boxes['predictions']:
            # Object = {}

            # Object['class '] = predciton['x']
            x1,y1,x2,y2 = predciton['x']-predciton['width']/2,predciton['y']-predciton['height']/2,predciton['x']+predciton['width']/2,predciton['y']+predciton['height']/2
            input_box= np.array([x1,y1,x2,y2])
            cate = predciton['class']
            # predictor.set_image(image)
            if use_center_of_mass== False:
                prompt_points.append([predciton['x'], predciton['y']])
            else:
                cx,cy = get_center_of_mass( input_box, image2)
                prompt_points.append([cx,cy])


        masks = []    
        masks_nosave = []
        skip_seg = []
        for j in range(len(prompt_points)):
            if j not in skip_seg:
                Object = {}

                # point_labels = np.zeros(len(prompt_points),dtype=int)
                # point_labels[j]=1
                predciton =  boxes['predictions'][j]
                x1,y1,x2,y2 = predciton['x']-predciton['width']/2,predciton['y']-predciton['height']/2,predciton['x']+predciton['width']/2,predciton['y']+predciton['height']/2
                input_box= np.array([x1,y1,x2,y2])
                Object['bbox'] =[x1,y1,x2,y2]

                point_labels = [1]
                prompt_points_temp = [prompt_points[j]]
                r0 = min(predciton['height'],predciton['width'])

                # for k in range(len(prompt_points)):
#                     if k!=j:
#                         if  prompt_points[k][0]>x1 and prompt_points[k][0]<x2 and  prompt_points[k][1]>y1 and prompt_points[k][1]<y2:
#                             # print('hehre')

#                             r = min(boxes['predictions'][k]['height'],boxes['predictions'][k]['width'])

#                             if boxes['predictions'][k]['class'] ==predciton['class']  and  \
#                             (prompt_points[k][0]-prompt_points[j][0])**2+(prompt_points[k][1]-prompt_points[j][1])**2<min(r0,r)**2 *0.1**2 :
#                                 skip_seg.append(k)
#                             else:
#                                 prompt_points_temp.append(prompt_points[k])
#                                 point_labels.append(0)



                mask, _, _ = predictor.predict(
                    point_coords=np.array(prompt_points_temp),
                    point_labels=np.array(point_labels),
                    box=None,
                    multimask_output=False,
                    )

                polygons = Mask(mask[0]).polygons()

                if len(polygons.points)>1:
                    lens = np.array([len(i) for i in polygons.points])
                    index = np.where(lens==max(lens))[0].item()
                    m = np.zeros(image.shape[:2])
                    img_origin = m.copy()
                    cv2.fillPoly(m,[np.array(polygons.points[index]).astype(np.int32)], 1)
                    m =  cv2.bitwise_xor(m, img_origin)
                    polygons_points = polygons.points[index].tolist()

                else:
                    m = np.zeros(image.shape[:2])
                    img_origin = m.copy()
                    cv2.fillPoly(m,[np.array(polygons.points[0]).astype(np.int32)], 1)
                    m =  cv2.bitwise_xor(m, img_origin)
                    polygons_points = polygons.points[0].tolist()

                Object['area'] = np.sum(m)   
                Object['score'] =predciton['confidence']
                Object['segmentation'] = polygons_points
                Object['category'] =predciton['class']
                masks.append(Object)
                Object_temp = copy.deepcopy(Object)
                Object_temp['segmentation_sam'] = m
                masks_nosave.append(Object_temp)
        pred_label['objects'] = masks
        #save to file ...
        pred_label_save = copy.deepcopy(pred_label)

        pred_label = json.dumps(pred_label)
        f = open(os.path.join(save_path_point+f'{image_name}.json'), 'w')
        f.write(pred_label)
        f.close()
    
    
    
    

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...


In [ ]:
# for sb in range(10):
#     if sb==2:
#         continue
#     print(sb)

In [ ]:
!cp -avr output_new_50/end2end output_new/end2end